### Deep Neural Network for Bound-Virtual Classfication

In this notebook we compile all the functions that are used to generate the training dataset for the bound-virtual enhancement classification. The dataset is prepared such that the input is the single-channel s-wave cross-section with enhancement at the threshold while the output is the corresponding label of the pole causing the enhancement. The output is described by

<br>
\begin{equation}
\begin{pmatrix}
\mbox{bound}   \\
\mbox{virtual} 
\end{pmatrix}
\end{equation}
<br>
and the input s-wave cross section is obtained by using the S-matrix parametrization

<br>
\begin{equation}
S(p)=e^{2i\delta_{bg}(p)}
\left(-\dfrac{p+i\gamma_{far}}{p-i\gamma_{far}}\right)
\left(-\dfrac{p+i\gamma_{near}}{p-i\gamma_{near}}\right)
\end{equation}
<br>
where $\delta_{bg}(p)$ is the background phase, $\gamma_{far}$ is the background virtual pole and $\gamma_{near}$ is the relevant near-threshold pole. The background phase shift is parametrized using the relation
<br>
\begin{equation}
\delta_{bg}(p)=\eta \tan^{-1}\left(\dfrac{p}{\Lambda_{bg}}\right).
\end{equation}
<br>
We start by importing the basic packages:

In [1]:
import math
import numpy as np
import random
import pickle

import matplotlib.pyplot as plt
from matplotlib.pyplot import figure

import sklearn
from sklearn.utils import shuffle

import chainer
from chainer.dataset import convert

We now define the global parameters used in generating the dataset. This includes the hadron masses, the center-of-mass energy $E_{cm}$ where the cross-section is to be plotted and the non-relativistic relation: $p=\sqrt{2\mu E_{cm}}$. The parameters of the background phase, which consists of $\eta$ and $\Lambda_{bg}$, are also specified. 

In [2]:
####################################
#Units in MeV
Nucleon = 938.9186795
Pion = 138.0394
Eta  = 547.862 

mu_NN   = 1/(1/Nucleon + 1/Nucleon)
mu_piN  = 1/(1/Pion + 1/Nucleon)
mu_etaN = 1/(1/Eta + 1/Nucleon)

    #Center-of-mass energy axis (in MeV)
Emax = 100.0 
E_Ndata = 200
Einput = np.linspace(Emax/E_Ndata, 
                     Emax, num=E_Ndata).astype(np.float32)


#choose system by specifying reduced mass (convert to fm^-1)
mu = mu_NN/(197.3)

#relative momentum (energy input is in MeV but p is in fm^-1)
#to be used in S-matrix evaluation
def p_rel(E):
    #E is in MeV
    #output p_rel is fm^-1
    return np.sqrt(2*mu*E/197.3)

#descriptive labels of network output
labelz = ['bound', 'virtual', 'resonance']

Nexpn = 10
#information about data generation
#expn = [-1, -2, -3, -4]
expn = np.random.uniform(low=-4,
                        high=-1,
                        size=Nexpn)
NLambda = 20
Ngamma  = 2*500
Nvirt   = 20

print('total number of data to be generated:', 
      len(expn)*NLambda*Ngamma*Nvirt)

#generate random values of background poles
#and near-threshold pole in units of 1/fm
#197.3 MeV*fm = hbar*c
Lamlow = 500  #in MeV/c
Lamhig = 700  #in MeV/c
Lambda = np.random.uniform(low=Lamlow/197.3,
                           high=Lamhig/197.3,
                           size=NLambda)
#to prevent a background to be closer to threshold than the bound state pole,
#we will vary gamma according to the value of Lambda

total number of data to be generated: 4000000


#### Generating the Training Dataset

The function
<br>
`amplitude2, labelout, ppole, vpole, BGprmtrs = gen_boundvirtual()`
<br> 
generates the bound-virtual classification dataset which contains the following:
- the input cross-section (s-wave amplituded)
- output label describing the nature of pole causing the enhancement
- the ner-threshold pole position in unit of fm${}^{-1}$
- background parameters and the position of far-threshold virtual pole    
<br>
<br> 
To simulate a near-threshold pole with a far virtual pole, we let $\gamma_{far}$ to be some multiple of $\gamma_{near}$, i.e. 
$|\gamma_{far}|\in\left(1.1|\gamma_{near}|,10|\gamma_{near}|\right)$. For the case of isolated near-threshold pole we let $|\gamma_{far}|\in\left(1.1|\Lambda_{bg}|,2.2|\Lambda_{bg}|\right)$.

In [3]:
def gen_boundvirtual():
    #define this functions as part of the module
    labelout   = []
    ppole      = []
    vpole      = []
    amplitude2 = []
    BGprmtrs   = []
    for xpindx in range(len(expn)):
        for lmindx in range(len(Lambda)):
            #Generate near-threshold pole
            
            #for shallow bound state poles
            gamlow = 0/197.3  #(in fm^-1) gives B.E. of 0.106 MeV
            gamhig = 200/197.3 #(in fm^-1) gives B.E. of 42.55 MeV
            gammapos = np.random.uniform(low=gamlow, high=gamhig, size=int(Ngamma/2))
            #for virtual state poles
            gamlow = 0/197.3            #in fm^-1
            gamhig = 0.9*Lambda[lmindx]  #in fm^-1    
            gammaneg = np.random.uniform(low=-gamhig, high=-gamlow, size=int(Ngamma/2))
            #combine bound and virtual
            gamma = np.concatenate((gammaneg, gammapos))
            
            for gamindx in range(len(gamma)):
                #The far virtual pole partner can be bigger than the
                #cut-off but cannot be smaller than the near-threshold pole
                #We just take the positive values below
                
                #for virtual pole near the near-threshold pole
                virt_near = np.random.uniform(low=1.1*abs(gamma[gamindx]),
                                              high=10.1*abs(gamma[gamindx]),
                                              size=int(Nvirt/2))
                #for virtual pole far from near-threshold pole
                virt_far  = np.random.uniform(low=1.1*abs(Lambda[lmindx]),
                                              high=2.1*abs(Lambda[lmindx]),
                                              size=int(Nvirt/2))
                #combine far virtual poles
                virtpole = np.concatenate((virt_near,virt_far))
                
                for virindx in range(len(virtpole)):
                    def PWAsqr(p):
                        #background phaseshift
                        sbg = np.exp(2*(1j)*expn[xpindx]*np.arctan(p/Lambda[lmindx]))
                        #with background virtual pole
                        sbg = -sbg*(p-(1j)*virtpole[virindx])/(p+(1j)*virtpole[virindx])
                        #The full S-matrix
                        smatrix = -sbg*(p+(1j)*gamma[gamindx])/(p-(1j)*gamma[gamindx])
                        #The partial wave amplitude
                        fmatrix = (smatrix-1)/(2*(1j)*p)
                        return abs(fmatrix*np.conj(fmatrix))
                    
                    #Classify cross-section according to the sign 
                    #of near-threshold pole
                    if gamma[gamindx]>0:
                        #label for near-threshold bound state
                        labelout.append(0)  
                    elif gamma[gamindx]<0:
                        #label for near-threshold virtual state
                        labelout.append(1)
                    
                    #relative momentum axis
                    prel = p_rel(Einput)
                    #cross-section
                    amplitude2.append(PWAsqr(prel))
                    
                    #auxillary data for checking:
                    #near-threshold pole value
                    ppole.append(gamma[gamindx]) #in fm^-1
                    #far virtual pole value
                    vpole.append(virtpole[virindx])  #in fm^-1
                    #background parameters
                    BGprmtrs.append([expn[xpindx],Lambda[lmindx]])
    
    #report the number of bound-virtual cross-sections
    labelz = [i for i, x in enumerate(range(len(labelout))) if labelout[x]==0]
    print('number of bound state:   ',len(labelz))
    labelz = [i for i, x in enumerate(range(len(labelout))) if labelout[x]==1]
    print('number of virtual state: ',len(labelz))
    
    return amplitude2, labelout, ppole, vpole, BGprmtrs

The function
<br>
`seecrosssec()`
<br>
gives the cross-section profile of a randmoly chosen sample from the bound-virtual dataset.
The parameters used to generate the data are also shown.

In [5]:
def seecrosssec():
    #chckind is a random integer
    #to call one of the generated data sample
    chckind = np.random.randint(0,len(labelout))
    font_set_size = 15
    plt.plot(Einput, amplitude2[chckind]*10,'o')
    plt.title('input data', fontsize=font_set_size)
    plt.xlabel('$E_{cm}$ (MeV)', fontsize=font_set_size)
    plt.xticks(fontsize=font_set_size)
    plt.ylabel('$|f_{\ell}(E)|^2$ ($mb$)', fontsize=font_set_size)
    plt.yticks(fontsize=font_set_size)
    plt.tight_layout()
    print('label:', labelz[labelout[chckind]])
    print('energy pole:',-ppole[chckind]**2/(2*mu)*197.3,'MeV')
    print('bg.v. pole:',-vpole[chckind]**2/(2*mu)*197.3,'MeV')
    print('eta:',BGprmtrs[chckind][0])
    print('Lambda:', BGprmtrs[chckind][1]*197.3,'MeV')
    return plt.show()

If you are now happy with the data generated, you may now export the dataset for future use.<br>
Enter the command 
<br>
`exportdata(amplitude2, labelout, ppole, vpole, BGprmtrs)`
<br>
to the console to export data in local directory.

In [6]:
def exportdata(amplitude2, labelout, ppole, vpole, BGprmtrs):
    #EXPORT generated data
    #Set delete=0 if you want to delete the generated data to free some space
    #Set delete=1 if you don't want to delete the generated data
    pickle.dump (ppole, open('ppole.pkl','wb'))
    pickle.dump (vpole, open('vpole.pkl','wb'))
    pickle.dump (amplitude2, open('amplitude2.pkl','wb'))
    pickle.dump (labelout, open('labelout.pkl','wb'))
    pickle.dump (BGprmtrs, open('BGprmtrs.pkl','wb'))
    print('export done')        
    return

In [34]:
del amplitude2, labelout, ppole, vpole, BGprmtrs

In [11]:
def get_testtrain(Nshuffle):
    #import prepared dataset
    inputtraining = pickle.load(open('amplitude2.pkl','rb'))
    outputtraining = pickle.load(open('labelout.pkl','rb'))
    print('size of training set:', len(inputtraining))
    print('number of nodes in input layer:',len(inputtraining[0]))
    #shuffle the imported data
    for ndx in range(Nshuffle):
        inputtraining, outputtraining = shuffle(inputtraining, outputtraining)
    #create input-output tuples that can be read by chainer
    dataset00 = chainer.datasets.TupleDataset(inputtraining, outputtraining)
    #split training set with testing set
    train, test = chainer.datasets.split_dataset(dataset00, int(0.8*len(inputtraining)), order=None)
    pickle.dump (train, open('chainer_train.pkl','wb'))
    pickle.dump (test, open('chainer_test.pkl','wb'))
    return train, test